<a href="https://colab.research.google.com/github/AreebaAleem/Digital_Image_Processing/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install opencv-python scikit-image scikit-learn keras


In [56]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from skimage.filters import gabor
from keras.preprocessing.image import ImageDataGenerator


In [57]:
import zipfile
import os

# List of your zip files
zip_files = ['valid.zip', 'test.zip']

# Extract each zip file
for zip_file in zip_files:
    try:
        zip_path = f'/content/{zip_file}'
        if os.path.exists(zip_path):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content/')
                print(f"Extracted {zip_file} successfully!")
        else:
            print(f"{zip_file} not found. Please check the file name and path.")
    except zipfile.BadZipFile:
        print(f"Failed to extract {zip_file}. The file may be corrupted or too large.")





Extracted valid.zip successfully!
Extracted test.zip successfully!


In [58]:

import zipfile
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your zip file
zip_path = '/content/drive/My Drive/train.zip'

# Extraction path
extract_to = '/content/train/'

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction Complete")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extraction Complete


In [59]:
# Preprocessing
def preprocess_image(image):
    # Resize image
    resized = cv2.resize(image, (64, 64))

    # Convert to grayscale
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

    # Convert to binary
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Convert to YCrCb color space
    ycrcb = cv2.cvtColor(resized, cv2.COLOR_BGR2YCrCb)

    return gray  # Return the processed image

    # Skin color range for YCrCb color space
    lower_skin = np.array([0, 133, 77], dtype=np.uint8)
    upper_skin = np.array([255, 173, 127], dtype=np.uint8)

    # Apply a skin mask
    skin_ycrcb = cv2.inRange(ycrcb, lower_skin, upper_skin)

    # Apply edge detection
    edges = cv2.Canny(skin_ycrcb, 100, 200)

    # Apply segmentation
    ret, segments = cv2.threshold(edges, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Apply filtering
    filtered = cv2.bilateralFilter(segments, 5, 75, 75)

    # Hand region detection
    contours, _ = cv2.findContours(filtered, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    if len(areas) > 0:
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        x, y, w, h = cv2.boundingRect(cnt)
        hand_region = filtered[y:y+h, x:x+w]
    else:
        hand_region = filtered

    # Background subtraction
    fgbg = cv2.createBackgroundSubtractorMOG2()
    fgmask = fgbg.apply(hand_region)

    # Apply your sharpening filter here
    # Your code here

    return fgmask

# Feature extraction
def extract_features(image):
    # Apply Gabor transformation
    gabor_feat, _ = gabor(image, frequency=0.6)
    return gabor_feat.flatten()



In [60]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

def load_dataset(image_path, label_path):
    images = []
    labels = []

    # Assuming label file that maps images to labels
    label_file_path = os.path.join(label_path, "labels.txt")  # Update as per your label storage format
    # Load labels
    with open(label_file_path, "r") as file:
        label_map = {line.split()[0]: int(line.split()[1]) for line in file.readlines()}

    # Load images and map labels
    for filename in os.listdir(image_path):
        img = cv2.imread(os.path.join(image_path, filename))
        if img is not None:
            images.append(img)
            # Assuming image filename matches the label key
            label = label_map.get(filename, -1)  # Default to -1 or any other flag value if label is missing
            labels.append(label)

    return images, labels

    images, labels = load_dataset("/content/train/train/images", "/content/train/train/labels")



def preprocess_image(image):
    # Resize and convert to grayscale as an example
    resized = cv2.resize(image, (64, 64))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    return gray

    processed_images = [preprocess_image(img) for img in images]


    # Data augmentation
    datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    augmented_images = datagen.flow(np.array(processed_images).reshape(len(processed_images), 64, 64, 1), np.array(labels))  # Ensure images are reshaped if needed

    # Split dataset into training and testing
    features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train a SVM classifier
    clf = SVC(gamma='auto')
    clf.fit(features_train, labels_train)

    # Test the model
    accuracy = clf.score(features_test, labels_test)
    print(f"Model accuracy: {accuracy}")